<a href="https://colab.research.google.com/github/musicjae/cs231n/blob/master/assignment2/Intro_to_TensorFLow_cs231n_Lecture_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 마운트 및 설명을 위한 이미지 불러오기 준비

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

## 1. 딥러닝 프레임워크  


### 1.1 왜 딥러닝 프레임워크를 사용해야 하는가? 

 
  
  - 이것을 사용하면 크고 복잡한 연산 그래프를 직접 만들지 않아도 된다.
  -  gradient를 자동적으로 계산해준다
  - 효과적으로 GPU를 다룰 수 있게 해준다.

In [ ]:
from IPython.display import Image

최근들어, 대세적인 딥러닝 프레임워크는 아래 그림과 같이 학계에서 산업계로 옮겨갔다

In [ ]:
Image('t1.png')

### 1.2 numpy와 대세적인 두 가지 딥러닝 프레임워크를 비교

In [ ]:
Image('t2.png')

3 가지 프레임 워크 상세 비교:  

- **Numpy case:** 직접 gradient를 구해줘야 하는 번거로움이 있다. 또한, gpu로 실행 못시킨다.  
- **TensorFlow case:** placeholder를 통해 순전파 계산을 쉽게 만들어주고, gradients 함수를 통해 grads
를 자동으로 구해준다. 또한, tf.device(‘/gpu(or cpu):0’)을 통해 gpu/cpu 선택 사용이 가능하다.
- **Pytorch case:** 위와 마찬가지로 gradients를 자동적으로 구해주는 기능이 있다. 마찬가지로, cuda()를 사용하여 gpu 사용이 가능하다.

### 1.3 텐서 플로우 자세히 알아보기  

EX의 Setting:  
- L2 loss 사용
-2 layers
-ReLU activation 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. Define Computational Graph

N,D,H = 64, 200, 100
#x = tf.placeholder(tf.float32, shape = (N,D))
#y = tf.placeholder(tf.float32, shape = (N,D))
x = tf.Variable(tf.random.normal((N,D))) 
y = tf.Variable(tf.random.normal((N,D))) 
w1 = tf.Variable(tf.random.normal((D,H))) # 병목 현상을 줄이기 위해 placeholder 대신 variable로 사용 
w2 = tf.Variable(tf.random.normal((H,D)))

h = tf.maximum(tf.matmul(x,w1),0) # hidden layer
y_pred = tf.matmul(h, w2) # 예측값 y

diff = y_pred - y # 오차
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))

#grad 함수를 쓰기 위해 설정

with tf.GradientTape() as g:

    g.watch(loss, (w1, w2))

grad_w1, grad_w2 = tf.g.gradient(loss, (w1, w2)) # gradeint 부분 수정 요구 (버젼 바뀜)

learning_rate = 1e-5
new_w1 = w1.assign(w1 - learning_rate * grad_w1) #grads 업데이트
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# 2. Run the graph many times

with tf.Session() as sess:
    sess.run(tf.global_varables_initializer()) # Variable를 그래프 내에 넣었기 때문에 이렇게 초기화해주어야 한다
    values = {x: np.random.randn(N,D),
              w1: np.random.randn(D,H),
              w2: np.random.randn(H,D),
              y: np.random.randn(N,D)}


    for t in range(50):
        out = sess.run([loss, grad_w1, grad_w2], feed_dict = values)
        loss_val, grad_w1_val, grad_w2_val = out
# 2.1 Train the network over and over.
        values[w1] -= learning_rate * grad_w1_val # 연산 그래프 외부에서 가중치 업데이트: 그래디언트 계산 => np_array로 가중치 업데이트
        values[w2] -= learning_rate * grad_w2_val

plt.plot(x,y)

#### 1.3.1 그래프 외부에서 가중치 업데이트

- **session:** 여러 번 그래프를 실행하려면 for 문을 사용하면 된다. sess.run()이 실행될 때마다 loss, grads를 구 해준다. session을 실행해주기 위해 np_array인 x, w1, w2, y를 넣어주면, 그 출력값으로 np_array 인 loss, grad1, grad2가 나온다.  
  

- **병목현상:** 그런데 문제가 있다. Forward pass 시 그래프가 실행될 때마다 가중치를 넣어feed주어야 한다. 그 래프가 한 번 실행될 때마다 가중치와 동일한 크기를 가진 grads가 반환된다. 이 반환이 함의하는 바 는 그래프 실행 마다 Tensorflow에서 numpy array인 가중치 행렬을 복사한다는 것이다. 문제는 이것이다: 이 경우에 GPU와 CPU 간 메모리 상의 데이터 교환이 있을 수 있는데, — 네트워크가 크고 가중치가 많은 경우에 — 이것은 아래 그림과 같이 GPU의 병목현상bottleneck을 일으킬 수 있다.   

- **variable**:텐서플로우에서는 이 문제를 해결할 방법이 있다. 위 코드 같이 placeholder로 저장하던 것을 Variable 로 저장해주는 것이다. variable은 매 시간마다 연산 그래프 내에 있는 변수이다.  

- **grads update:** 또한, 위 코드에서 나타나듯이, 우리는 이제 그래프 내부에서 assign 함수를 이용하여 w1, w2를 업데 이트 시킬 수 있다.  
  
- **init:** session()을 통해 학습을 할 때, 그래프 내에 있는 변수들을 초기화시켜주기 위해, tf.global_variables_intialier()을 사용한다. 그 뒤에 그래프를 계속 반복하여 실행시켜준다.

In [ ]:
Image('t3.png',width=400)

하지만 위 코드로 학습을 시킨다면, 아래 그림과 같이 손실이 내려가지 않는다.

In [ ]:
Image('t4.png')

### 1.3.2 더미 노드 추가

위 곤경을 해소할 방안은 바로 그래프에 더미 노드를 추가하는 다소 tricky한 방식이다. 우리는 이 허 위 데이터 의존성fake data dependencies을 이용하여, 이 더미 노드가 새로운 w1, w2를 업데이트해준다고 말할 수 있다. 이 더미 노드는 어떤 것도 반환하지 않지만, 이 의존성 때문에 우리는 값을 업 데이트 할 수 있고, 그래서 실제로 업데이트 연산을 수행하는 것이다.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. Define Computational Graph

N,D,H = 64, 200, 100

x = tf.Variable(tf.random.normal((N,D))) 
y = tf.Variable(tf.random.normal((N,D))) 
w1 = tf.Variable(tf.random.normal((D,H))) 
w2 = tf.Variable(tf.random.normal((H,D)))

h = tf.maximum(tf.matmul(x,w1),0) 
y_pred = tf.matmul(h, w2)

diff = y_pred - y 
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))

#grad 함수를 쓰기 위해 설정

with tf.GradientTape() as g:

    g.watch(loss, (w1, w2))

grad_w1, grad_w2 = tf.g.gradient(loss, (w1, w2)) # gradeint 부분 수정 요구 (버젼 바뀜)

learning_rate = 1e-5
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)
updates = tf.group(new_w1, new_w2) # 더미 노드 추가
# 2. Run the graph many times

with tf.Session() as sess:
    sess.run(tf.global_varables_initializer()) # Variable를 그래프 내에 넣었기 때문에 이렇게 초기화해주어야 한다
    values = {x: np.random.randn(N,D),
              w1: np.random.randn(D,H),
              w2: np.random.randn(H,D),
              y: np.random.randn(N,D)}


    for t in range(50):
        out = sess.run([loss, updates, grad_w1, grad_w2], feed_dict = values) # updates 더미 노드 추가
        loss_val, grad_w1_val, grad_w2_val = out
# 2.1 Train the network over and over.
        values[w1] -= learning_rate * grad_w1_val 
        values[w2] -= learning_rate * grad_w2_val

plt.plot(x,y)

### 1.3.3 최적화

또한 우리는 위 함수를 이용하여 grad를 구하고 W를 업데이트 하기 위해 최적화를 사용할 수 있다. 이 함수는 환상적이다. 왜냐하면 이 w1, w2 변수가 디폴트에 의해 학습될 수 있다고 나타나기 때문 에, 이 optimizer.minimize 함수는 그 안에서 내부적으로 w1, w2에 대한 손실의 그래디언트를 계산 하는 그래프에 노드들을 추가해주기 때문이다.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. Define Computational Graph

N,D,H = 64, 200, 100

x = tf.Variable(tf.random.normal((N,D))) 
y = tf.Variable(tf.random.normal((N,D))) 
w1 = tf.Variable(tf.random.normal((D,H))) 
w2 = tf.Variable(tf.random.normal((H,D)))

h = tf.maximum(tf.matmul(x,w1),0) 
y_pred = tf.matmul(h, w2)

diff = y_pred - y 
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))


learning_rate = 1e-5

optimizer = tf.optimizers.SGD(1e-5) # 최적화 추가 (버젼 바뀜으로 수정 요구)
updates = optimizer.minimize(loss, var_list=[w1,w2]) 
# 2. Run the graph many times

with tf.Session() as sess:
    sess.run(tf.global_varables_initializer()) 
    values = {x: np.random.randn(N,D),
              w1: np.random.randn(D,H),
              w2: np.random.randn(H,D),
              y: np.random.randn(N,D)}


    for t in range(50):
        out = sess.run([loss, updates, grad_w1, grad_w2], feed_dict = values) # updates 더미 노드 추가
        loss_val, grad_w1_val, grad_w2_val = out
# 2.1 Train the network over and over.
        values[w1] -= learning_rate * grad_w1_val 
        values[w2] -= learning_rate * grad_w2_val

plt.plot(x,y)

### 1.3.4 손실

또한 우리는 위 함수를 이용하여 grad를 구하고 W를 업데이트 하기 위해 최적화를 사용할 수 있다. 이 함수는 환상적이다. 왜냐하면 이 w1, w2 변수가 디폴트에 의해 학습될 수 있다고 나타나기 때문 에, 이 optimizer.minimize 함수는 그 안에서 내부적으로 w1, w2에 대한 손실의 그래디언트를 계산 하는 그래프에 노드들을 추가해주기 때문이다.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers as opim

# 1. Define Computational Graph

N,D,H = 64, 200, 100

x = tf.Variable(tf.random.normal((N,D))) 
y = tf.Variable(tf.random.normal((N,D))) 
w1 = tf.Variable(tf.random.normal((D,H))) 
w2 = tf.Variable(tf.random.normal((H,D)))

h = tf.maximum(tf.matmul(x,w1),0) 
y_pred = tf.matmul(h, w2)

loss = tf.losses.mean_squared_error(y_pred,y)
with tf.compat.v1.Session() as sess:
    sess.run(opim.RSMprop()) 
    values = {x: np.random.randn(N,D),
              w1: np.random.randn(D,H),
              w2: np.random.randn(H,D),
              y: np.random.randn(N,D)}


    for t in range(50):
        out = sess.run([loss, updates, grad_w1, grad_w2], feed_dict = values) # updates 더미 노드 추가
        loss_val, grad_w1_val, grad_w2_val = out
# 2.1 Train the network over and over.
        values[w1] -= learning_rate * grad_w1_val 
        values[w2] -= learning_rate * grad_w2_val

plt.plot(x,y)